In [ ]:
# Initialisation
import sys; sys.path.insert(0,'..')
import lpse_utils.write_files as wf
from lpse_utils.extract_data import get_metrics,get_fields
import os
import numpy as np
%load_ext autoreload
%autoreload 2
os.system('> lpse.parms')
try:
  os.mkdir('data')
except:
  print('Data directory exists.')
show_atts = True

LPSE user guide contains additional options to those facilitated by current input classes

# Job Control

In [ ]:
jc = wf.job_control()
jc.version = '3.2.11' 
jc.seed = 1 # 0 for random, otherwise fixed seed
jc.resources.heartbeatInterval = 0.1 # minutes
jc.write(show_atts)

# Gridding

In [ ]:
gr = wf.gridding()
gr.grid.sizes = 100 # microns
gr.grid.nodes = 4000
gr.write(show_atts)

# Components

In [ ]:
cm = wf.components()
cm.laser.enable = 'true'
cm.raman.enable = 'true'
cm.lw.enable = 'true'
cm.write(show_atts)

# Temporal Control

In [ ]:
tc = wf.temporal_control()
tc.simulation.samplePeriod = 0.05 # ps
tc.simulation.time.end = 2 # ps
tc.write(show_atts)

# I/O Control

In [ ]:
# Specifying a filename turns on output
io = wf.io_control()
io.grid.downSampleFactors = 4 # Spatial downsampling per dimension
fp = './data/lpse.' # File prefix
io.lw.save.pots = fp + 'pots'
io.laser.save.E0.z = fp + 'E0_z'
io.laser.save.S0.x = fp + 'S0_x'
io.raman.save.E0.z = fp + 'E1_z'
io.raman.save.S0.x = fp + 'S1_x'
io.raman.save.E0.__dict__['2'] = None
io.write(show_atts)

# Physical Parameters

In [ ]:
pp = wf.physical_parameters()
pp.physical.Z = 3.5
pp.physical.Te = 4.5
pp.physical.Ti = 4.5
pp.physical.MiOverMe = 11848.8
pp.lw.envelopeDensity = 0.15
pp.densityProfile.shape = 'exp'
pp.densityProfile.NminOverNc = 0.14
pp.densityProfile.NmaxOverNc = 0.17
pp.densityProfile.NminLocation = '-50 0 0'
pp.densityProfile.NmaxLocation = '50 0 0'
pp.write(show_atts)

# Light Control

In [ ]:
lc = wf.light_control()
lc.laser.pumpDepletion.SRS = 'true'
lc.laser.solver = 'fd'
lc.laser.evolution.Labc = 3
lc.laser.evolution.Loff = 1
lc.raman.sourceTerm.lw.enable = 'true'
lc.raman.solver = 'fd'
lc.raman.evolution.Labc = 3
lc.write(show_atts)

# Light Sources

In [ ]:
# Laser light
beams = 1 # Number of beamlets
inten = ['2e+14'] # Intensities [W/cm^2]
phase = [0] # Phases [degrees]
polar = [90] # Polarisations [degrees]
fshift = [0] # Frequency shifts from envelope frequency
group = [0] # Beamlet groups
direc = ['1 0 0'] # Direction (converted to unit vector internally)
source = ['min.x'] # Boundary source
offset = ['0 0 0'] # Offset from boundary centre [microns]
width = [10] # Half-width at 1/e of super gaussian [microns]
sgord = [4] # Order of super gaussian

# Write
specs = [inten,phase,polar,fshift,group,\
        direc,source,offset,width,sgord]
light_input('laser',beams,specs)

# Langmuir Wave Parameters

In [ ]:
lwc = wf.lw_control()
lwc.lw.SRS = 'true'
lwc.lw.spectral.dt = 0.002 # ps
lwc.lw.Labc = 3 # microns
lwc.lw.noise.enable = 'true'
lwc.lw.noise.isCalculated = 'true'
lwc.lw.noise.amplitude = 1.67772
lwc.lw.collisionalDampingRate = 0.1
lwc.lw.__dict__['collisionalDampingRate.isCalculated'] = 'false'
lwc.write(show_atts)

# Ion Acoustic Wave Parameters

In [ ]:
iawc = wf.iaw_control()
iawc.write(show_atts)

# Instrumentation

In [ ]:
ins = wf.instrumentation()
ins.metrics.enable = 'true'
ins.metrics.file = fp + 'metrics'
ins.metrics.samplePeriod = 0.01 # ps
ins.write()

# Run case

In [ ]:
!{'mpirun -np 1 ../../bin/lpse_cpu --parms=lpse.parms'}

# Visualise and extract datasets

In [ ]:
# Extract metrics data
mkeys, mdat = get_metrics(plot=True)
print(mkeys)

In [ ]:
# Extract fields data
innerkys = ['time','x','data']
outerkys = [field_fnames[i].replace(f_prefix,'') for i in range(len(field_fnames))]
field_dicts = {i:{j:None for j in innerkys} for i in outerkys}
for i in range(len(field_fnames)):
    field_dicts[outerkys[i]] = get_fields(field_fnames[i],field_dicts[outerkys[i]],dsample,plot=True)

In [ ]:
# Laser light transmission at final timestep
icell = 50
ocell = -50
linx = field_dicts['S0_x']['x'][icell]
lin = np.real(field_dicts['S0_x']['data'][-1,icell]) 
loutx = field_dicts['S0_x']['x'][ocell]
lout = np.real(field_dicts['S0_x']['data'][-1,ocell])
enl = 1-lout/lin
print(f'Laser inner Poynting flux (at {linx:0.1f} um)' \
      + f' at sim end: {lin:0.3e} W/cm^2')
print(f'Laser outer Poynting flux (at {loutx:0.1f} um)' \
      + f' at sim end: {lout:0.3e} W/cm^2')
print(f'This is an energy loss of {enl:.2%}')

In [ ]:
# Raman light energy gain at final timestep
icell = -50
ocell = 50
rinx = field_dicts['S1_x']['x'][icell]
rin = np.real(field_dicts['S1_x']['data'][-1,icell]) 
routx = field_dicts['S1_x']['x'][ocell]
rout = np.real(field_dicts['S1_x']['data'][-1,ocell])
eng = rout/rin - 1
lenl = abs(rout-rin)/(lin-lout)
print(f'Raman inner Poynting flux (at {rinx:0.1f} um)' \
      + f' at sim end: {rin:0.3e} W/cm^2')
print(f'Raman outer Poynting flux (at {routx:0.1f} um)' \
      + f' at sim end: {rout:0.3e} W/cm^2')
print(f'This is an energy gain of {eng:.2%}')
print(f'It represents {lenl:.2%} of the laser energy loss')